In [1]:
import requests
from pandas.io.json import json_normalize
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash.dependencies import Input, Output
from plotly.subplots import make_subplots


#Geo map of Cases per country globally
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)


In [2]:
url = 'https://api.covid19api.com/summary'
r = requests.get(url)
dictr = r.json()
header = dictr['Countries']
df = json_normalize(header)
df = df.drop(df.index[[0,224]])
df = df.drop(columns = 'Slug')
df = df.sort_values(by=['TotalConfirmed'], ascending=False)
df.head()

/home/tshepo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning:

pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead



,Country,CountryCode,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
234,United States of America,US,24513,604736,2227,25747,4281,47763,2020-04-15T23:22:23Z
206,Spain,ES,2442,172541,300,18056,2777,67504,2020-04-15T23:22:23Z
107,Italy,IT,2972,162488,602,21067,1695,37130,2020-04-15T23:22:23Z
80,Germany,DE,1287,131359,100,3294,3900,68200,2020-04-15T23:22:23Z
233,United Kingdom,GB,5275,94845,782,12129,19,323,2020-04-15T23:22:23Z


In [3]:
#Summing per column
tot_cases = df.TotalConfirmed.sum()
nw_cases = df.NewConfirmed.sum()
tot_deaths = df.TotalDeaths.sum()
new_deaths = df.NewDeaths.sum()
tot_recover = df.TotalRecovered.sum()
new_recover = df.NewRecovered.sum()
diff = tot_cases - nw_cases
diff2 = tot_recover - new_recover
diff3 = tot_deaths - new_deaths 

#Separating columns

#Corona  cases: Total and New
df_tot_cases = df.loc[:,['Country','TotalConfirmed']].sort_values(by='TotalConfirmed', ascending=False)
df_new_cases = df.loc[:,['Country','NewConfirmed']]

#Death Cases: Total and New 
df_tot_death = df.loc[:,['Country','TotalDeaths']]
df_new_death = df.loc[:,['Country','NewDeaths']]

#Recovery cases: Total and new
df_tot_recover = df.loc[:,['Country','TotalRecovered']]
df_new_recover = df.loc[:,['Country','NewRecovered']]
                      
#Sorting cases by total                     
df_cases = df_tot_cases.merge(df_new_cases, on='Country').sort_values(by=['TotalConfirmed'],
                                                                      ascending=False)
df_death = df_tot_death.merge(df_new_death, on='Country').sort_values(by=['TotalDeaths'],
                                                                      ascending=False)
df_recover = df_tot_recover.merge(df_new_recover, on='Country').sort_values(by=['TotalRecovered'], 
                                                                            ascending=False)


In [4]:
style_table = {
    'maxHeight': '50ex',
    'overflowY': 'scroll',
    'width': '30%',
    'minWidth': '30%',
}
style_cell = {
    'fontFamily': 'Open Sans',
    'textAlign': 'center',
    'height': '30px',
    'padding': '10px 22px',
    'whiteSpace': 'inherit',
    'overflow': 'hidden',
                'textOverflow': 'ellipsis',
}
style_cell_conditional = [
    {
        'if': {'column_id': 'State'},
        'textAlign': 'left'
    },
]
style_header = {
    'fontWeight': 'bold',
    'backgroundColor': "#3D9970",
    'fontSize': '16px'
}
style_data_conditional = [
    {
        'if': {'row_index': 'odd'},
        'backgroundColor': 'rgb(248, 248, 248)'
    }]
style_table = {
    'maxHeight': '50ex',
    'overflowY': 'scroll',
    'width': '100%',
    'minWidth': '100%',
}

In [5]:
a = df.TotalConfirmed.sum()-df.NewConfirmed.sum()

In [6]:
# # Style sheet
# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# # App
# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# app.layout = html.Div(children=[
#         #header
    
#     html.Div([
#         html.Div([
#             html.Img(src="https://dcassetcdn.com/design_img/3666062/781819/781819_21156990_3666062_e336b60a_image.png",className='four columns')
            
#         ], className="four columns"),
        

#         html.Div([
#             html.H3('Latest Covid 19 Update', style={
#                                 'fontFamily': 'Open Sans',
#                                 'textAlign': 'center',
#                             }),
             
#         ], className="four columns"),
        
#         html.Div([
#             html.H3('#StayAtHome', style={
#                                 'fontFamily': 'Open Sans',
#                                 'textAlign': 'right',
#                             }),
            
#         ], className="four columns")
#     ], className="row"),
    
#     dcc.Tabs(id='tabs-example', value='tab', children=[
#         dcc.Tab(label='Global Map', value='tab-1'),
#         dcc.Tab(label='South African cases', value='tab-2'),
#         dcc.Tab(label='Analysis', value='tab-3'),
#     ]),
#     html.Div(id='tabs-example-content')
# ])




# def render_content(tab):
#     '''Display the table and the graph sideby
#     side when the respective tab is selected'''

#     if tab == 'tab-1':
#         return html.Div([
#             html.Div([
       
#                 html.H3('band', style={
#                                     'fontFamily': 'Open Sans',
#                                     'textAlign': 'center',
#                                 })

#         ], className="row"),

#         #VISUALISATIONS
#          html.Div([
#             html.Div([
#                 html.H3('visualisation'),
#                 html.Div([
#                     html.Div(
#                         [
#                             html.P("Total confirmed cases"),
#                             html.H6(
#                                 id="well_text",
#                                 className="info_text"
#                             )
#                         ],
#                         id="wells",
#                         className="pretty_container"
#                     )
#                 ], className="row"),
#                 html.Div([

#                 html.H3('Cases by country'),
#                     dash_table.DataTable(
#                             id='table',
#                             data=df_df_tot_cases.to_dict("rows"),
#                             columns=[{"name": i, "id": i}
#                                      for i in df_tot_cases.columns],
#                             style_table=style_table,
#                             style_cell=style_cell,
#                             style_data_conditional=style_data_conditional,
#                             style_header=style_header,
#                             style_cell_conditional=style_cell_conditional,
#                         )

#         ], className="row"),

#             ], className="two columns"),
#              html.Div([
#                 html.H3('visualisation'),
#                 html.Div([

#                 html.H3('World Map')

#         ], className="row"),

#             ], className="six columns"),

#             html.Div([
#                 html.H3('visualisation'),


#                 html.Div([
#                     html.Div([
#                         html.H3('Total Recoveries'),

#                         ], className="six columns"),

#                     html.Div([
#                         html.H3('Total Deaths'),

#                         ], className="six columns"),
#         ], className="row")

#             ], className="four columns"),
#         ], className="row"),

#         #fOOTER
#          html.Div([
#             html.Div([
#                 html.H3('Footer'),

#             ], className="six columns"),

#             html.Div([
#                 html.H3('Footer'),

#             ], className="six columns"),
#         ], className="row")
#         ])
        
 

    
    
        
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
#     html.Div([
#             html.Div([
#                 html.Div([
#                     html.H1(
#                         children='Table',
#                         style={
#                             'textAlign': 'center',
#                             'color': 'green'
#                         }), dash_table.DataTable(
#                         id='table',
#                         data=df_sum_state.to_dict("rows"),
#                         columns=[{"name": i, "id": i}
#                                  for i in df_sum_state.columns],
#                         style_table=style_table,
#                         style_cell=style_cell,
#                         style_data_conditional=style_data_conditional,
#                         style_header=style_header,
#                         style_cell_conditional=style_cell_conditional,
#                     )
#                 ], className="four columns"),
#                 html.Div([
#                     html.H3('Fires per state in Brazil',
#                             style={
#                                 'textAlign': 'center',
#                                 'color': 'green'}),
#                     dcc.Graph(
#                         id='Graph1',
#                         figure={
#                             'data': [
#                                 {'x': df.state, 'y': df.number, 'type': 'bar', 'name': 'state', 'marker': {'color': [
#                                     '#b50000', '#e63535', '#fa8989'], 'size': 9, 'line': {'width': 1, 'color': 'orange'}}},
#                             ]
#                         }
#                     )
#                 ], className="eight columns"),
#             ], className="row")
#         ])



# if __name__ == '__main__':
#     app.run_server()


In [7]:
# # layout
# app.layout = html.Div([
#       #Band
#      html.Div([
       
#             html.H3('band', style={
#                                 'fontFamily': 'Open Sans',
#                                 'textAlign': 'center',
#                             })
         
#     ], className="row"),
    
#     #VISUALISATIONS
#      html.Div([
#         html.Div([
#             html.H3('visualisation'),
#             html.Div([
       
#                 html.H3('Total confirmed cases')
         
#             ], className="row"),
#             html.Div([
       
#             html.H3('Cases by country')
         
#     ], className="row"),
            
#         ], className="two columns"),
#          html.Div([
#             html.H3('visualisation'),
#             html.Div([
       
#             html.H3('World Map')
         
#     ], className="row"),
            
#         ], className="six columns"),

#         html.Div([
#             html.H3('visualisation'),
            
            
#             html.Div([
#                 html.Div([
#                     html.H3('Total Recoveries'),
                    
#                     ], className="six columns"),

#                 html.Div([
#                     html.H3('Total Deaths'),

#                     ], className="six columns"),
#     ], className="row")
            
#         ], className="four columns"),
#     ], className="row"),
    
#     #fOOTER
#      html.Div([
#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),

#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),
#     ], className="row"),
    
# ])






In [8]:
# #header
    
#     html.Div([
#         html.Div([
#             html.H3('Header'),
            
#         ], className="four columns"),
        

#         html.Div([
#             html.H3('header'),
             
#         ], className="four columns"),
        
#         html.Div([
#             html.H3('header'),
            
#         ], className="four columns")
#     ], className="row"),
    
        
#     #NavBar
       
#      html.Div([
#         html.Div([
#             html.H3('nav-bar'),
           
#         ], className="four columns"),
#         html.Div([
#             html.H3('nav-bar'),
           
#         ], className="four columns"),

#         html.Div([
#             html.H3('nav-bar'),
            
#         ], className="four columns"),
#     ], className="row"),
    
#       #Band
#      html.Div([
       
#             html.H3('band')
         
#     ], className="row"),
    
#     #VISUALISATIONS
#      html.Div([
#         html.Div([
#             html.H3('visualisation'),
#             html.Div([
       
#                 html.H3('Total confirmed cases')
         
#             ], className="row"),
#             html.Div([
       
#             html.H3('Cases by country')
         
#     ], className="row"),
            
#         ], className="two columns"),
#          html.Div([
#             html.H3('visualisation'),
#             html.Div([
       
#             html.H3('World Map')
         
#     ], className="row"),
            
#         ], className="six columns"),

#         html.Div([
#             html.H3('visualisation'),
            
            
#             html.Div([
#                 html.Div([
#                     html.H3('Total Recoveries'),
                    
#                     ], className="six columns"),

#                 html.Div([
#                     html.H3('Total Deaths'),

#                     ], className="six columns"),
#     ], className="row")
            
#         ], className="four columns"),
#     ], className="row"),
    
#     #fOOTER
#      html.Div([
#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),

#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),
#     ], className="row"),


### Dashboard pages template

In [9]:
# # Style sheet
# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# # App
# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# # layout
# app.layout = html.Div([
    
#     #header
    
#     html.Div([
#         html.Div([
#             html.H3('Header'),
            
#         ], className="four columns"),
        

#         html.Div([
#             html.H3('header'),
             
#         ], className="four columns"),
        
#         html.Div([
#             html.H3('header'),
            
#         ], className="four columns")
#     ], className="row"),
    
        
#     #NavBar
       
#      html.Div([
#         html.Div([
#             html.H3('nav-bar'),
           
#         ], className="four columns"),
#         html.Div([
#             html.H3('nav-bar'),
           
#         ], className="four columns"),

#         html.Div([
#             html.H3('nav-bar'),
            
#         ], className="four columns"),
#     ], className="row"),
    
#       #Band
#      html.Div([
       
#             html.H3('band')
         
#     ], className="row"),
    
#     #VISUALISATIONS
#      html.Div([
#         html.Div([
#             html.H3('visualisation'),
#             html.Div([
       
#                 html.H3('Total confirmed cases')
         
#             ], className="row"),
#             html.Div([
       
#             html.H3('Cases by country')
         
#     ], className="row"),
            
#         ], className="two columns"),
#          html.Div([
#             html.H3('visualisation'),
#             html.Div([
       
#             html.H3('World Map')
         
#     ], className="row"),
            
#         ], className="six columns"),

#         html.Div([
#             html.H3('visualisation'),
            
            
#             html.Div([
#                 html.Div([
#                     html.H3('Total Recoveries'),
                    
#                     ], className="six columns"),

#                 html.Div([
#                     html.H3('Total Deaths'),

#                     ], className="six columns"),
#     ], className="row")
            
#         ], className="four columns"),
#     ], className="row"),
    
#     #fOOTER
#      html.Div([
#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),

#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),
#     ], className="row"),
    
# ])

# if __name__ == '__main__':
#     app.run_server()


In [10]:
# total cases indicator
fig1 = go.Figure(go.Indicator(
    
    value = tot_cases,
    delta = {'reference': diff},
    gauge = {
        'axis': {'visible': False}},
    domain = {'row': 0, 'column': 0}))

fig1 = fig1.update_layout(
    template = {'data' : {'indicator': [{
        'title': {'text': "Total Cases"},
        'mode' : "number+delta+gauge",
        }]
                         }})
fig1.show()

In [11]:
#Summing per column
tot_cases = df.TotalConfirmed.sum()
nw_cases = df.NewConfirmed.sum()
tot_deaths = df.TotalDeaths.sum()
new_deaths = df.NewDeaths.sum()
tot_recover = df.TotalRecovered.sum()
new_recover = df.NewRecovered.sum()
diff = tot_cases - nw_cases

#Recoveries indicator
fig2 = go.Figure(go.Indicator(
    mode = "number+delta",
    value = tot_recover,
    delta = {'reference': diff2},
    domain = {'row': 0, 'column': 1}))
fig2 = fig2.update_layout(
    template = {'data' : {'indicator': [{
        'title': {'text': "Recoveries"}
        }]
                         }})
fig2.show()

In [12]:
# total cases indicator
fig3 = go.Figure(go.Indicator(
    
    value = tot_deaths,
    delta = {'reference': diff3},
    gauge = {
        'axis': {'visible': False}},
    domain = {'row': 0, 'column': 0}))

fig3 = fig3.update_layout(
    template = {'data' : {'indicator': [{
        'title': {'text': "Total Deaths"},
        'mode' : "number+delta+gauge",
        }]
                         }})
fig3.show()

In [13]:
#Style sheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# App
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
        #header
    
    html.Div([
        html.Div([
            html.Img(src="https://dcassetcdn.com/design_img/3666062/781819/781819_21156990_3666062_e336b60a_image.png",className='four columns')
            
        ], className="four columns"),
        

        html.Div([
            html.H3('Latest Covid 19 Update', style={
                                'fontFamily': 'Open Sans',
                                'textAlign': 'center',
                            }),
             
        ], className="four columns"),
        
        html.Div([
            html.H3('#StayAtHome', style={
                                'fontFamily': 'Open Sans',
                                'textAlign': 'right',
                            }),
            
        ], className="four columns")
    ], className="row"),
    
    dcc.Tabs(id='tabs-example', value='tab', children=[
        dcc.Tab(label='Global Map', value='tab-1'),
        dcc.Tab(label='South African cases', value='tab-2'),
        dcc.Tab(label='Analysis', value='tab-3'),
    ]),
    html.Div(id='tabs-example-content')
])

In [ ]:
@app.callback(Output('tabs-example-content', 'children'),
              [Input('tabs-example', 'value')])
def render_content(tab):

    if tab == 'tab-1':
        return html.Div([
            html.Div([

                # VISUALISATIONS

                html.Div([
                      dcc.Graph(
                          id='figure',
                            figure=go.Figure(fig1)
                        ),
                html.Div([
                    dash_table.DataTable(
                            id='table',
                            data=df_tot_cases.to_dict("rows"),
                            columns=[{"name": i, "id": i}
                                     for i in df_tot_cases.columns],
                            style_table=style_table,
                            style_cell=style_cell,
                            style_data_conditional=style_data_conditional,
                            style_header=style_header,
                            style_cell_conditional=style_cell_conditional,
                        )
                ])
                    
            ], className="row"),

                ], className="four columns"),
            
            html.Div([
                    dcc.Graph(
                          id='figure',
                            figure=go.Figure(fig2)
                        ),
                    html.Div([
                        dcc.Tabs(id='tabs-examples', value='tabs', children=[
                            dcc.Tab(label='Total Cases', value='tab-1.'),
                            dcc.Tab(label='Active Cases', value='tab-2.'),
                            dcc.Tab(label='Solved Cases', value='tab-3.'),
                            dcc.Tab(label='Recovery Rate', value='tab-4.'),
                            dcc.Tab(label='Case Fatality Rate', value='tab-5.')
                        ]),
                     html.Div(id='tabs-example-contents'),
            ], className="row"),
                ], className="four columns"),

                html.Div([
                     dcc.Graph(
                          id='figure',
                            figure=go.Figure(fig3)
                        ),


                    html.Div([
                       
            ], className="row")

                ], className="four columns"),
            ], className="row")

            # fOOTER
        html.Div([
                html.Div([
                    html.H3('Footer'),

                ], className="six columns"),

                html.Div([
                    html.H3('Footer'),

                ], className="six columns"),
            ], className="row")
  

    elif tab == 'tab-2':
        return html.P("You got it T")
    else:
        return html.H1('O ntja saan!!!')

    # @app.callback(Output('tabs-example-content', 'children'),
#               [Input('tabs-example', 'value')])
# def render_content(tab):
    
#     if tab == 'tab-1':
#         return html.H1(
#         children='Confirmed Cases by Country',
#         style={
#             'textAlign': 'center',
#             'color': 'maroon'
#         }),dcc.Graph(
#         id='figure',
#         figure=
#           go.Figure(fig)
#         )
#     elif tab == 'tab-2':   
#         return html.H1(
#         children='Active Cases by Country',
#         style={
#             'textAlign': 'center',
#             'color': 'maroon'
#         }),dcc.Graph(
#         id='figure',
#         figure=
#           go.Figure(fig1)
#         )
           
#     elif tab == 'tab-3':   
#         return html.H1(
#         children='Solved Cases by Country',
#         style={
#             'textAlign': 'center',
#             'color': 'maroon'
#         }), dcc.Graph(
#         id='figure',
#         figure=
#           go.Figure(fig2)
#         )
#     elif tab == 'tab-4':
#          return html.H1(
#         children='Recovery Rate per country',
#         style={
#             'textAlign': 'center',
#             'color': 'maroon'
#         }), dcc.Graph(
#         id='figure',
#         figure=
#           go.Figure(fig3)
#         )
#     else:
#         return html.H1(
#         children='Fatality Rate per Country',
#         style={
#             'textAlign': 'center',
#             'color': 'maroon'
#         }),dcc.Graph(
#         id='figure',
#         figure=
#           go.Figure(fig4)
#         )



if __name__ == '__main__':
    app.run_server()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Apr/2020 01:37:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 01:37:47] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 01:37:47] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 01:37:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 01:37:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 01:38:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 01:38:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 01:38:54] "POST /_dash-update-component HTTP/1.1" 200 -


In [ ]:
go.Indicator(
    mode = "number+delta",
    value = 300,
    domain = {'row': 0, 'column': 1}))

In [ ]:
# # Style sheet
# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# # App
# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# # layout
# app.layout = html.Div([
    
#     #header
    
#     html.Div([
#         html.Div([
#             html.H3('Header'),
            
#         ], className="four columns"),
        

#         html.Div([
#             html.H3('header'),
             
#         ], className="four columns"),
        
#         html.Div([
#             html.H3('header'),
            
#         ], className="four columns")
#     ], className="row"),
    
        
#     #NavBar
       
#      html.Div([
#         html.Div([
#             html.H3('nav-bar'),
           
#         ], className="four columns"),
#         html.Div([
#             html.H3('nav-bar'),
           
#         ], className="four columns"),

#         html.Div([
#             html.H3('nav-bar'),
            
#         ], className="four columns"),
#     ], className="row"),
    
#       #Band
#      html.Div([
       
#             html.H3('band')
         
#     ], className="row"),
    
#     #VISUALISATIONS
#      html.Div([
#         html.Div([
#             dcc.Graph(                    
#                 id='figure',
#                 figure=
#                   go.Figure(fig1)
#             ),
#             html.Div([
       
#                 html.H3('Total confirmed cases')
         
#             ], className="row"),
#             html.Div([
       
#             html.H3('Cases by country')
         
#     ], className="row"),
            
#         ], className="two columns"),
#          html.Div([
#             html.H3('visualisation'),
#             html.Div([
       
#             html.H3('World Map')
         
#     ], className="row"),
            
#         ], className="six columns"),

#         html.Div([
#             html.H3('visualisation'),
            
            
#             html.Div([
#                 html.Div([
#                     html.H3('Total Recoveries'),
                    
#                     ], className="six columns"),

#                 html.Div([
#                     html.H3('Total Deaths'),

#                     ], className="six columns"),
#     ], className="row")
            
#         ], className="four columns"),
#     ], className="row"),
    
#     #fOOTER
#      html.Div([
#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),

#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),
#     ], className="row"),
    
# ])

# if __name__ == '__main__':
#     app.run_server()


In [ ]:
#Total confirmed world map
data = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z = df['TotalConfirmed'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(155,155,155)',
 width = 4,
 )  ),
 colorbar = dict(
 title = 'Number of cases'
 )
 ) ]
layout = dict(
 autosize=True,
    
)
fig = go.Figure(data = data, layout = layout)

# Active cases world map

data1 = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z = df['Active Cases'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(255,255,255)',
 width = 4,
 )  ),
 colorbar = dict(
 title = 'Active cases'
 )
 ) ]
layout = dict(
 autosize=True,
    
)
fig1_ = go.Figure(data = data1, layout = layout)

#Closed cases world map
data2 = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z =df['Closed Cases'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(245, 197, 66)',
 width = 4,
 )  ),
 colorbar = dict(
 title = 'Closed Cases'
 )
 ) ]
layout = dict(
 autosize=True,
    
)
fig2_ = go.Figure(data = data2, layout = layout)

#Recovery Rate World Map

data3 = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z =df['Recovery Rate'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(455,455,455)',
 width = 4,
 )  ),
 colorbar = dict(
 title = 'Recovery Rate (%)'
 )
 ) ]
layout = dict(
 autosize=True,
    
)
fig3_ = go.Figure(data = data3, layout = layout)


#Case fatality rate world map

data4 = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z =df['Case Fatality Rate'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(555,555,555)',
 width = 4,
 )  ),
 colorbar = dict(
 title = 'Case Fatality Rate (%)'
 )
 ) ]
layout = dict(
 autosize=True,
    
)
fig4_ = go.Figure(data = data4, layout = layout)
